# Paquetes

In [2]:
# !pip install sciann --quiet

In [3]:
# !pip install mlflow --quiet

In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
# import os

# os.chdir('/content/drive/MyDrive/Colab Notebooks/TFM/Experimento Placa')

In [ ]:
#Paquetes para modelos
import sciann as sn
import numpy as np
from sciann.utils.math import diff, sign, sin, exp
from sciann.utils import math as scmath
import tensorflow as tf 

#MlFlow
import mlflow
from mlflow import log_metric, log_param


from itertools import product

import time

%load_ext autotime

#  Preprocesamiento de datos

## Modelo numérico

In [ ]:
def lee_u(u_file):
    values = []
    times = []
    fd = open(u_file,'r')
    for tupla in fd.read().split('\n\n'):
        if tupla != '':
            tiempo, datos = tupla.split('\n')
            times.append(float(tiempo))
            vector = []
            for value in datos.split('\t'):
                    if value != '':
                        vector.append(float(value))
            values.append(vector)
    values = np.array(values)
    fd.close()
    return times, values

def lee_mesh(mesh_file):
    fd = open(mesh_file)
    for batch in fd.read().split('\n\n'):
        if batch[0:8]=='Vertices':
            vertices = batch
    fd.close()

    coordenadas = []

    for vertice in vertices.split('\n'):
        if len(vertice.split(' ')) == 3:
            coordenadas.append([float(vertice.split(' ')[0]),float(vertice.split(' ')[1])])
    coordenadas = np.array(coordenadas)
    return coordenadas

def preparadatos(u_file,mesh_file):
    datos = lee_u(u_file)
    mesh = lee_mesh(mesh_file)
    
    sec = datos[0][0]
    fullmesh = np.insert(mesh,2,sec,axis=1)
    
    for i in range(1,len(datos[0])):
        sec = datos[0][i]
        fullmesh = np.append(fullmesh,np.insert(mesh,2,sec,axis=1),axis=0)
    
    valores = datos[1].flatten()
    
    return fullmesh,valores

from scipy.interpolate import griddata

## Mallado del test (estático)



In [ ]:
denspt = 10 # Densidad de puntos de evaluación de los funcionales
xmin,xmax=-2,2 # Límites en x
ymin,ymax =-2,2 # Límites en y
tmin,tmax=0,10 # Límites en tiempo t
xrange=xmax-xmin
yrange=ymax-ymin
trange=tmax-tmin

xx = np.linspace(xmin, xmax, xrange*denspt) 
yy = np.linspace(ymin, ymax, yrange*denspt)
tt = np.linspace(tmin, tmax, trange*denspt)

grid = np.meshgrid(xx,yy,tt)

x_test, y_test, t_test = grid

In [4]:
np.product(x_test.shape)

160000

time: 16 ms (started: 2021-08-08 11:16:24 +02:00)


In [5]:
#Archivos de datos
u_file = 'u.txt'
mesh_file = 'ofen'
#Predicciones del numérico
puntos, valores = preparadatos(u_file,mesh_file)
print('Datos preparados')
u_pred_Num = griddata(puntos, valores, (x_test, y_test, t_test) , method='nearest')

Datos preparados
time: 21.3 s (started: 2021-08-08 11:16:24 +02:00)


## Wrappers en forma de funciones


In [6]:
# Activación para los bordes

def activacion(coord,location,direction):
    return (1-sign(direction*(location-coord)))/2



time: 0 ns (started: 2021-08-08 11:16:45 +02:00)


## Constantes del problema

In [7]:
TOL = 0.05 # Esta constante se usa para que los bordes tomen valores y se activen las funciones de contorno
conveccion = 1. # Coeficiente de pérdida de calor en los bordes y=1 e y=-1
conductividad = 1. # Capacidad conductora de calor del material de la placa
capCal = 1. # Capacidad de retención de calor del material de la placa
focoTemp = 10. # Temperatura del foco de calor
tempExt = 5. # Temperatura del exterior
tempInicial = 0. # Temperatura inicial de la placa
sigma = 0.4 #Amplitud del foco de calor

time: 0 ns (started: 2021-08-08 11:16:45 +02:00)


# Mallado de parámetros de experimentación

In [1]:
trainpt = [2]
u_Structure = [3*[100]]
u_Activator = ['tanh']
epochs = [2]
loss = ['mse']
optimizer = ['SGD']
batch_size = [2000]



configuraciones = list(product(trainpt,u_Structure,u_Activator,epochs,loss,optimizer,batch_size))
configuraciones = [list(configuracion) for configuracion in configuraciones]
configuraciones_finalizadas = list([])

print(len(configuraciones))

NameError: name 'product' is not defined

In [9]:
# configuraciones_finalizadas = configuraciones[:25]
# np.save('configuraciones_finalizadas',configuraciones_finalizadas,allow_pickle=True)

time: 0 ns (started: 2021-08-08 11:16:45 +02:00)


In [10]:
configuraciones_finalizadas = np.load('configuraciones_finalizadas.npy',allow_pickle=True).tolist()
configuraciones_restantes = [configuracion for configuracion in configuraciones if configuracion not in configuraciones_finalizadas]
print(len(configuraciones_restantes))

17
time: 15 ms (started: 2021-08-08 11:16:45 +02:00)


# Loop

In [ ]:
# resultados  = []
for configuracion in configuraciones_restantes:
    with mlflow.start_run():
        
        print('Experimentando con:' + str(configuracion))
        
        #Parámetros
        #################
        trainpt = configuracion[0]
        u_Structure = configuracion[1]
        u_Activator = configuracion[2]
        epochs = configuracion[3]
        loss = configuracion[4]
        optimizer = configuracion[5]
        batch_size = configuracion[6]
        #################

        #Log de parámetros
        #################
        mlflow.log_param('Density', trainpt)
        mlflow.log_param('u_Structure', u_Structure)
        mlflow.log_param('u_Activator', u_Activator)
        mlflow.log_param('Epochs', epochs)
        mlflow.log_param('Loss', loss)
        mlflow.log_param('Optimizer', optimizer)
        mlflow.log_param('Batch size', batch_size)
        #################
        #Variables del modelo
        #################
        x = sn.Variable("x") 
        y = sn.Variable('y')
        t = sn.Variable('t')
        u = sn.Functional("u", [x,y,t], u_Structure ,u_Activator)
        #################
    
        #Construcción del mallado
        #################
        

        # Mallado (es una malla en tres dimensiones)
        denspt = trainpt # Densidad de puntos de evaluación de los funcionales
        xmin,ymin,xmax,ymax=-2,-2,2,2 # Límites en x e y
        tmin,tmax=0,10 # Límites en tiempo t
        xrange,yrange=xmax-xmin,ymax-ymin
        trange=tmax-tmin

        # Mallado (es una malla en tres dimensiones)
        x_data, y_data, t_data = np.meshgrid(
            np.linspace(xmin, xmax, xrange*denspt), 
            np.linspace(ymin, ymax, yrange*denspt),
            np.linspace(tmin,tmax,trange*denspt)
        )

        x_data = x_data.flatten()[:,None]
        y_data = y_data.flatten()[:,None]
        t_data = t_data.flatten()[:,None]

        numpoints =  trainpt**3*xrange*yrange*trange
        
        mlflow.log_param('Num_Points', numpoints)
        #################

        #Restricciones
        #################
        
        #Aislamiento en los bordes x=-2 y x=2
        L2 = conveccion * diff(u,x)
        d2 = sn.constraints.Data(L2)
        #Indices de los bordes
        indAislamiento = np.where(np.abs(x_data)+TOL>2)[0][:,None]
        datos2 = [(indAislamiento,'zeros')]
        
        #Temperatura constante en los bordes y=-2 e y=2
        indConveccion = np.where(np.abs(y_data)+TOL > 2)[0][:,None]
        d3 = sn.constraints.Data(u)
        datos3 = [(indConveccion,tempExt)]
        
        #Temperatura inicial
        indInicial = np.where(t_data < TOL)[0][:,None]
        valInicial = tempExt/2 * np.abs(y_data[indInicial,0])
        d4 = sn.constraints.Data(u)
        datos4 = [(indInicial,valInicial)]
  
        #Temperatura foco
        radio = np.sqrt(x_data[:,0]**2+y_data[:,0]**2)
        indCentro = np.where(radio<sigma)[0]
        indT_desde_05 = np.where(t_data[:,0]>5)[0]
        indCentro_desde_05 = np.intersect1d(indCentro,indT_desde_05)[:,None]
        valCentro = 5 + 5 *  np.exp(-(x_data[indCentro_desde_05,0]**2+y_data[indCentro_desde_05,0]**2)/(2*(sigma)**2))/np.sqrt((2*np.pi*sigma**2))*(1/(1+np.exp(-4*(t_data[indCentro_desde_05,0]-6))))
        d5 = sn.constraints.Data(u)
        datos5 = [(indCentro_desde_05,valCentro)]

        #Este funcional explica el flujo del calor a lo largo de la placa (2 dimensiones)

        ecCalor = capCal*diff(u,t) - conductividad *( diff(u,x,order=2) + diff(u,y,order=2))
        L1 = sn.constraints.PDE(ecCalor)
        fullset = np.arange(numpoints)[:,None]
        indSinFoco = np.setdiff1d(fullset,indCentro_desde_05,assume_unique=True)
        indSinFoco = np.setdiff1d(indSinFoco,indConveccion)

        datos1 = [(indSinFoco,'zeros')]
 
        #Agregamos restricciones
        datos = datos1+datos2+datos3+datos4+datos5

        #################
        #Construcción del modelo
        m = sn.SciModel ([x,y,t], [L1,d2,d3,d4,d5], loss,optimizer)

        #Entrenamiento del modelo
        start = time.process_time()
        history = m.train([x_data, y_data, t_data], datos, epochs =epochs,verbose=0,
                        batch_size=batch_size)
        TEntrenamiento = time.process_time() - start

        #Testeo del modelo:

        start2 = time.process_time()
        u_pred_sciann =  u.eval(m, [x_test, y_test, t_test])
        TTesteo = time.process_time() - start2

        MAE = np.mean(np.abs(u_pred_Num-u_pred_sciann))*160
        RMSE = np.sqrt(np.mean((u_pred_Num-u_pred_sciann)**2))*160

        pred = m.model.predict([np.reshape(x_test,(-1,)),np.reshape(y_test,(-1,)),np.reshape(t_test,(-1,))])

        funcional = np.mean(np.abs(np.array(pred[1])))
        
        MAX = np.max(np.abs(u_pred_Num-u_pred_sciann))
        
        mlflow.log_metric('L1',MAE)
        mlflow.log_metric('L2',RMSE)
        mlflow.log_metric('MaxError',MAX)
        mlflow.log_metric('Funcional',funcional)
        mlflow.log_metric('Tiempo_Entrenamiento',TEntrenamiento)
        mlflow.log_metric('Tiempo_Evaluacion',TTesteo)
        
        configuraciones_finalizadas.append(configuracion)
        np.save('configuraciones_finalizadas',configuraciones_finalizadas)

2021/08/08 11:16:49 WARNING mlflow.tracking.context.git_context: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh()

All git commands will error until this is rectified.

This initial warning can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|none|n|0: for no warning or exception
    - warn|w|warning|1: for a printed warning
    - error|e|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet



Experimentando con:[15, [100, 100, 100], 'tanh', 2000, 'mae', 'Nadam', 20000]

Total samples: 540000 
Batch size: 20000 
Total batches: 27 

